In [1]:
from src import visualization as vis
from src import pre_process
from src import cnn

import os
import cv2
import matplotlib.pyplot as plt
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

c:\Users\saraa\miniconda3\envs\tfm\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def napari():
    vis.visualize_images("D:/TFM/Microscopy/video/3288-1.czi")
    vis.napari_view_splited_channels()

# napari()

In [3]:
# 1. Creation of dataframes for each channel independently and all together
blue_df, green_df, red_df, df = pre_process.create_df("D:/TFM/Microscopy/video/copies/channels/")

def save_df(df_list, path_list, index = True):
    for df, path in zip(df_list, path_list): 
        df.to_csv(path, index = index)
        print(f'Dataframe has been saved to {path}')

df_list = [blue_df, green_df, red_df, df]
path_list = ["C:/Users/saraa/TFM/Microscopy/data/blue","C:/Users/saraa/TFM/Microscopy/data/green", "C:/Users/saraa/TFM/Microscopy/data/red","C:/Users/saraa/TFM/Microscopy/data/rgb"]
save_df(df_list, path_list)

['D:/TFM/Microscopy/video/copies/channels/blue', 'D:/TFM/Microscopy/video/copies/channels/green', 'D:/TFM/Microscopy/video/copies/channels/red']
Dataframe has been saved to C:/Users/saraa/TFM/Microscopy/data/blue
Dataframe has been saved to C:/Users/saraa/TFM/Microscopy/data/green
Dataframe has been saved to C:/Users/saraa/TFM/Microscopy/data/red
Dataframe has been saved to C:/Users/saraa/TFM/Microscopy/data/rgb


In [4]:
green_df

,File_path,Image_id,Channel,Sample,Duplicates,Timepoints,Label
0,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0000,green,3288,1,0000,1
1,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0001,green,3288,1,0001,1
2,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0002,green,3288,1,0002,1
3,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0003,green,3288,1,0003,1
4,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0004,green,3288,1,0004,1
...,...,...,...,...,...,...,...
196,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0196,green,3288,1,0196,1
197,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0197,green,3288,1,0197,1
198,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0198,green,3288,1,0198,1
199,D:/TFM/Microscopy/video/copies/channels/green\...,3288-1g0199,green,3288,1,0199,1


In [5]:
def masks_df(input_stack, type, output_df):
    # 2. Generate masks for each microscopy image stack 
    cnn.mask(input_stack, type)
    # 3. Generate mask dataframe
    maskdf = pre_process.mask_df(output_df)
    return maskdf

input_stack = "D:\\TFM\\Microscopy\\video\\3288-1-AP-OP.czi"
type = "czi"
output_df = 'C:/Users/saraa/TFM/mask/nuclei/'
nucleiM = masks_df(input_stack, type, output_df)

(1, 201, 3, 1, 940, 940, 1)


100%|██████████| 603/603 [00:49<00:00, 12.29it/s]


In [6]:
red = red_df.drop(columns = ["Sample", "Duplicates"])
red


,File_path,Image_id,Channel,Timepoints,Label
0,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0000,red,0000,0
1,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0001,red,0001,0
2,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0002,red,0002,0
3,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0003,red,0003,0
4,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0004,red,0004,0
...,...,...,...,...,...
196,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0196,red,0196,0
197,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0197,red,0197,0
198,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0198,red,0198,0
199,D:/TFM/Microscopy/video/copies/channels/red\32...,3288-1r0199,red,0199,0


In [7]:
df_list = [nucleiM]
path_list = ["C:/Users/saraa/TFM/Microscopy/data/nucleiM"]
save_df(df_list, path_list)

Dataframe has been saved to C:/Users/saraa/TFM/Microscopy/data/nucleiM


In [8]:
Xtrain, Xtest, Ytrain, Ytest = pre_process.train_valid_split(red, nucleiM)

In [9]:
def split_shutil_train_val():
    # 1. Creating folders 
    data_path = "D:/TFM/Microscopy/video/copies/img/"
    folder_names= ["train_folder", "val_folder"]
    pre_process.createfolders(data_path,folder_names)

    data_path = "D:/TFM/Microscopy/video/copies/masks/"
    folder_names= ["train_folder", "val_folder"]
    pre_process.createfolders(data_path,folder_names)
    
    # 2. Splitting images data
    import shutil
    dfs = [Xtrain, Xtest]
    paths = ["D:/TFM/Microscopy/video/copies/img/train_folder/", "D:/TFM/Microscopy/video/copies/img/val_folder/"]
    for path, df in zip(paths, dfs):
        try: 
            pre_process.move_images(df, path)
        except shutil.Error:
            print(f"The files already exist in the destination path: {path}")
    
    # 3. Splitting masks data
    dfs = [Ytrain, Ytest]
    paths = ["D:/TFM/Microscopy/video/copies/masks/train_folder/", "D:/TFM/Microscopy/video/copies/masks/val_folder/"]
    for path, df in zip(paths, dfs):
        try: 
            pre_process.move_images(df, path)
        except shutil.Error:
            print(f"The files already exist in the destination path: {path}")  

# split_shutil_train_val()

Folder train_folder already exists at: D:/TFM/Microscopy/video/copies/img/train_folder
Folder val_folder already exists at: D:/TFM/Microscopy/video/copies/img/val_folder
Folder train_folder already exists at: D:/TFM/Microscopy/video/copies/masks/train_folder
Folder val_folder already exists at: D:/TFM/Microscopy/video/copies/masks/val_folder


In [10]:
def create_shutil_labels():
    # 1. Creating folders 
    train_path = "D:/TFM/Microscopy/video/copies/img/train_folder"
    val_path = "D:/TFM/Microscopy/video/copies/img/val_folder"
    folder_names= ["red", "green", "blue"]
    pre_process.createfolders(train_path,folder_names)
    pre_process.createfolders(val_path,folder_names)

    train_path = "D:/TFM/Microscopy/video/copies/masks/train_folder"
    val_path = "D:/TFM/Microscopy/video/copies/masks/val_folder"
    folder_names= ["red", "green", "blue"]
    pre_process.createfolders(train_path,folder_names)
    pre_process.createfolders(val_path,folder_names)
    
    # 2. Creating labels for training and testing data
    Xtrain["new_file_path"] = "D:/TFM/Microscopy/video/copies/img/train_folder/" + Xtrain["Image_id"] + ".tif"
    Xtrain["new_file_path"] = Xtrain["new_file_path"].astype(str)
    Xtest["new_file_path"] = "D:/TFM/Microscopy/video/copies/img/val_folder/" + Xtest["Image_id"] + ".tif"
    Xtest["new_file_path"] = Xtest["new_file_path"].astype(str)
    
    Ytrain["new_file_path"] = "D:/TFM/Microscopy/video/copies/masks/train_folder/" + Ytrain["Image_id"] + ".png"
    Ytrain["new_file_path"] = Ytrain["new_file_path"].astype(str)
    Ytest["new_file_path"] = "D:/TFM/Microscopy/video/copies/masks/val_folder/" + Ytest["Image_id"] + ".png"
    Ytest["new_file_path"] = Ytest["new_file_path"].astype(str)
    
    # 3. Save dataframe
    Xtrain.to_csv("C:/Users/saraa/TFM/Microscopy/data/img_training", index = True)
    Xtest.to_csv("C:/Users/saraa/TFM/Microscopy/data/img_testing", index = True)
    Ytrain.to_csv("C:/Users/saraa/TFM/Microscopy/data/mask_training", index = True)
    Ytest.to_csv("C:/Users/saraa/TFM/Microscopy/data/mask_testing", index = True)

    # 4. Splitting data for images 
    img_train_path_red = "D:/TFM/Microscopy/video/copies/img/train_folder/red"
    # train_path_green = "D:/TFM/Microscopy/video/copies/train_folder/green"
    # train_path_blue = "D:/TFM/Microscopy/video/copies/train_folder/blue"
    pre_process.images_class(Xtrain, img_train_path_red)    

    img_val_path_red = "D:/TFM/Microscopy/video/copies/img/val_folder/red"
    # val_path_green = "D:/TFM/Microscopy/video/copies/val_folder/green"
    # val_path_blue = "D:/TFM/Microscopy/video/copies/val_folder/blue"
    pre_process.images_class(Xtest, img_val_path_red)

    # 5. Splitting data for masks
    mask_train_path_red = "D:/TFM/Microscopy/video/copies/masks/train_folder/red"
    # train_path_green = "D:/TFM/Microscopy/video/copies/train_folder/green"
    # train_path_blue = "D:/TFM/Microscopy/video/copies/train_folder/blue"
    pre_process.images_class(Ytrain, mask_train_path_red)    

    mask_val_path_red = "D:/TFM/Microscopy/video/copies/masks/val_folder/red"
    # val_path_green = "D:/TFM/Microscopy/video/copies/val_folder/green"
    # val_path_blue = "D:/TFM/Microscopy/video/copies/val_folder/blue"
    pre_process.images_class(Ytest, mask_val_path_red)

    return img_train_path_red, img_val_path_red, mask_train_path_red, mask_val_path_red
    
# img_train_path_red, img_val_path_red, mask_train_path_red, mask_val_path_red = create_shutil_labels()    

Folder red already exists at: D:/TFM/Microscopy/video/copies/img/train_folder\red
Folder green created at: D:/TFM/Microscopy/video/copies/img/train_folder\green
Folder blue created at: D:/TFM/Microscopy/video/copies/img/train_folder\blue
Folder red already exists at: D:/TFM/Microscopy/video/copies/img/val_folder\red
Folder green created at: D:/TFM/Microscopy/video/copies/img/val_folder\green
Folder blue created at: D:/TFM/Microscopy/video/copies/img/val_folder\blue
Folder red already exists at: D:/TFM/Microscopy/video/copies/masks/train_folder\red
Folder green created at: D:/TFM/Microscopy/video/copies/masks/train_folder\green
Folder blue created at: D:/TFM/Microscopy/video/copies/masks/train_folder\blue
Folder red already exists at: D:/TFM/Microscopy/video/copies/masks/val_folder\red
Folder green created at: D:/TFM/Microscopy/video/copies/masks/val_folder\green
Folder blue created at: D:/TFM/Microscopy/video/copies/masks/val_folder\blue


In [11]:
from src import cnn, pre_process
from keras.layers import Input
from keras.optimizers import Adam

img_train_path = "D:/TFM/Microscopy/video/copies/img/train_folder/"
img_val_path = "D:/TFM/Microscopy/video/copies/img/val_folder/"
mask_train_path = "D:/TFM/Microscopy/video/copies/masks/train_folder/"
mask_val_path = "D:/TFM/Microscopy/video/copies/masks/val_folder/"

img_gen, valid_img_gen, mask_gen, valid_mask_gen = pre_process.get_generator(img_train_path, img_val_path, 
                                                                             mask_train_path, mask_val_path)

Found 160 images belonging to 3 classes.
Found 41 images belonging to 3 classes.
Found 160 images belonging to 3 classes.
Found 41 images belonging to 3 classes.


In [12]:
pre_process.combine_generators(img_gen, mask_gen)

<generator object combine_generators at 0x000002FAB1100F90>

In [13]:
input_layer = Input((900, 900, 1))
# 1. Initialize the model 
model = cnn.unet_model(input_layer, 16)

# 2. Fit the model 
model.compile(optimizer = Adam(lr = 1e-3), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 900, 900, 16), dtype=tf.float32, name=None), name='conv2d_1/Relu:0', description="created by layer 'conv2d_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 450, 450, 32), dtype=tf.float32, name=None), name='conv2d_3/Relu:0', description="created by layer 'conv2d_3'")
KerasTensor(type_spec=TensorSpec(shape=(None, 225, 225, 64), dtype=tf.float32, name=None), name='conv2d_5/Relu:0', description="created by layer 'conv2d_5'")
KerasTensor(type_spec=TensorSpec(shape=(None, 112, 112, 128), dtype=tf.float32, name=None), name='conv2d_7/Relu:0', description="created by layer 'conv2d_7'")
KerasTensor(type_spec=TensorSpec(shape=(None, 112, 112, 128), dtype=tf.float32, name=None), name='conv2d_transpose/BiasAdd:0', description="created by layer 'conv2d_transpose'")
KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 64), dtype=tf.float32, name=None), name='cropping2d/strided_slice:0', description="created by layer 'cropping2d'")
KerasTensor(type_s

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 448, 448, 32), (None, 449, 449, 32)]